In [6]:
import pandas as pd
import numpy as np
#import tensorflow as tf
from sklearn import preprocessing 

In [7]:
data_url='http://archive.ics.uci.edu/ml/machine-learning-databases/postoperative-patient-data/post-operative.data'
names = ['L-CORE', 'L-SURF', 'L-02', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL', 'COMFORT', 'decision']
df = pd.read_csv(data_url, names=names)
df[:5]

,L-CORE,L-SURF,L-02,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [8]:
#오류나는 Y 데이터 전처리(같은 A값인데 다르게 나옴)
df['decision'][3]=df['decision'][4]
df[:5]

,L-CORE,L-SURF,L-02,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,10,S
2,high,low,excellent,high,stable,stable,mod-stable,10,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,10,A


In [9]:
#COMPORT 데이터 전처리 '?'값 평균값으로 처리

int_data = df['COMFORT'] [df['COMFORT'] != '?'].astype('int64')

average=sum(int_data)/len(int_data)

df['COMFORT'] [df['COMFORT'] == '?']=str(average)

for c in ['L-CORE', 'L-SURF', 'L-02', 'L-BP', 'SURF-STBL', 'CORE-STBL', 'BP-STBL', 'decision','COMFORT']:
    df[c] = df[c].astype('category')

In [10]:
#y데이터와 x데이터로 나눔
y_data=df.pop('decision')
#x_data=pd.get_dummies(df).values



In [14]:
y_dummies=pd.get_dummies(y_data)


#print(df.corrwith(y_data).sort_values(ascending=True))
print('---------------S----------------')
print(df.corrwith(y_dummies["S"]).sort_values(ascending=True))
print('---------------A----------------')
print(df.corrwith(y_dummies["A"]).sort_values(ascending=True))
print('---------------I----------------')
print(df.corrwith(y_dummies["I"]).sort_values(ascending=True))

---------------S----------------


TypeError: ("unsupported operand type(s) for /: 'str' and 'int'", 'occurred at index L-CORE')

In [6]:
df=pd.get_dummies(df)
df[:5]

,L-CORE_high,L-CORE_low,L-CORE_mid,L-SURF_high,L-SURF_low,L-SURF_mid,L-02_excellent,L-02_good,L-BP_high,L-BP_low,...,CORE-STBL_stable,CORE-STBL_unstable,BP-STBL_mod-stable,BP-STBL_stable,BP-STBL_unstable,COMFORT_05,COMFORT_07,COMFORT_10,COMFORT_10.9425287356,COMFORT_15
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_data[:,0]=min_max_scaler.fit_transform(x_data[:,0])

x_data[:5,0]

C:\Users\진홍\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\진홍\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


array([ 0.,  0.,  1.,  0.,  0.])

In [8]:
#LogisticRegression 모델 사용 후 성능 평가

from sklearn.cross_validation import KFold,ShuffleSplit
from sklearn.linear_model import LogisticRegression

average_perpomance=np.zeros(100)
cv  = ShuffleSplit(len(y_data),  n_iter=100, test_size=0.3, random_state=0)

y_data = np.array(y_data)
x_data = np.array(x_data)

for num,(train_index, test_index) in enumerate(cv) :
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    clf = LogisticRegression()
    clf.fit(x_train, y_train)
    result = clf.predict(x_test)
   
    average_perpomance[num]=(sum(result == y_test) / len(y_test))

print(sum(average_perpomance)/len(average_perpomance))

0.664444444444


In [9]:
# x 데이터에 Y의 S 라벨이 가장 영향을 많이 받으면서, L-BP_low와 CORE_STBL_unstable 컬럼이 가장 상관도가 높음
#COMFORT 변수는 drop시길 예정

y_dummies=pd.get_dummies(y_data)

print('---------------S----------------')
print(df.corrwith(y_dummies["S"]).sort_values(ascending=True))
print('---------------A----------------')
print(df.corrwith(y_dummies["A"]).sort_values(ascending=True))
print('---------------I----------------')
print(df.corrwith(y_dummies["I"]).sort_values(ascending=True))

---------------S----------------
COMFORT_10.9425287356   -1.119785e-01
L-BP_low                -1.119785e-01
L-BP_high               -1.066004e-01
CORE-STBL_stable        -1.063248e-01
L-SURF_high             -9.842731e-02
BP-STBL_mod-stable      -9.504969e-02
COMFORT_10              -7.479576e-02
BP-STBL_unstable        -6.528618e-02
CORE-STBL_mod-stable    -6.392028e-02
L-CORE_high             -3.335452e-02
L-CORE_mid              -2.449530e-02
L-02_excellent          -2.347408e-02
SURF-STBL_stable        -2.789540e-18
SURF-STBL_unstable       2.789540e-18
L-SURF_low               1.869894e-02
L-02_good                2.347408e-02
COMFORT_15               5.746398e-02
L-CORE_low               5.746398e-02
L-SURF_mid               6.043672e-02
COMFORT_05               7.954545e-02
BP-STBL_stable           1.373891e-01
CORE-STBL_unstable       1.410190e-01
L-BP_mid                 1.459918e-01
COMFORT_07               1.757808e-01
dtype: float64
---------------A----------------
COMFORT

In [10]:
#feature engineering (추가 & 삭제)
import copy

df2=df.copy()

#y와 상관도가 높은 변수 생성
df2['HIGH_REL'] = df['CORE-STBL_unstable']*100000+df['L-BP_low']*10000000

df2=df2.values
df2=df2[:,:-5]

In [11]:
x_data=df2
y_data.shape, x_data.shape

((90,), (90, 20))

In [12]:
#feature engineering 후 재평가

from sklearn.cross_validation import KFold,ShuffleSplit
# from sklearn.naive_bayes import MultinomialNB

average_perpomance=np.zeros(100)
cv  = ShuffleSplit(len(y_data),  n_iter=100, test_size=0.3, random_state=0)

y_data = np.array(y_data)
x_data = np.array(x_data)

for num,(train_index, test_index) in enumerate(cv) :
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    clf = LogisticRegression()
    clf.fit(x_train, y_train)
    result = clf.predict(x_test)
   
    average_perpomance[num]=(sum(result == y_test) / len(y_test))

print(sum(average_perpomance)/len(average_perpomance))

0.675555555556


In [13]:
#decsion tree 모델 사용 후 성능 평가

from sklearn import tree

average_perpomance=np.zeros(100)
cv  = ShuffleSplit(len(y_data),  n_iter=100, test_size=0.3, random_state=0)

y_data = np.array(y_data)
x_data = np.array(x_data)

for num,(train_index, test_index) in enumerate(cv) :
    x_train, x_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(x_data, y_data)
    result = clf.predict(x_test)
   
    average_perpomance[num]=(sum(result == y_test) / len(y_test))

print(sum(average_perpomance)/len(average_perpomance))



0.901851851852
